<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ab8e607cc233e963ad223d789f482bce5cd040659f274cf1e28dbaf9aaad7539
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 14:41:44
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.70 L (-1.18%)
Current PnL: -21.75 L (-14.25%)
CY Booked + Current PnL: -9.10 L (-5.96%)
-------------------
Total profit:  2.05 L
Total loss:  -23.80 L
-------------------
Total Booked + Current PnL: 17.88 L (14.23%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.86%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.73 L (62.13%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.66,64.0,X-LC,5.76,230927.0,20561.0,8221.0,0.49,9.77,3.56,13.68,37.0,2.50,1.63,27.91,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,69.0,M-LC,10.76,174416.0,33576.0,10918.0,1.23,23.84,6.26,31.59,66.0,3.08,1.23,56.64,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.20,53.0,M-SC,1.76,87574.0,-13203.0,13294.0,-0.04,-13.10,15.18,0.09,245.0,-0.99,0.62,15.05,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.91,61.0,X-MC,5.60,149865.0,13732.0,16845.0,-0.73,10.09,11.24,22.47,91.0,0.82,1.06,24.38,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-12.47,33.0,X-MC,2.69,86550.0,-4128.0,19032.0,-1.74,-4.55,21.99,16.44,101.0,-0.22,0.61,14.70,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-9.14,69.0,X-LC,4.63,229258.0,-22510.0,75724.0,4.77,-8.94,33.03,21.13,27.0,-0.30,1.62,23.32,X40,ATH,PAINTS
5,ANGELONE,3033.00,20.92,68.0,X-SC,5.85,214036.0,23030.0,34674.0,3.64,12.06,16.20,30.21,157.0,0.66,1.51,35.46,X40N,NTT,FINANCE
25,GREENPANEL,537.00,211.37,44.0,M-SC,6.78,139080.0,-43998.0,122975.0,1.93,-24.03,88.42,43.14,230.0,-0.36,0.98,30.82,XY24,NTT,MISC
68,SONACOMS,806.63,-30.74,70.0,M-SC,6.87,86196.0,-14963.0,54967.0,1.81,-14.79,63.77,39.54,202.0,-0.27,0.61,21.55,AR,ATH,AUTO
14,BSOFT,836.99,-14.41,57.0,H-SC,11.34,96135.0,-44518.0,117294.0,1.78,-31.65,122.01,51.74,131.0,-0.38,0.68,11.23,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3305.41,32.0,M-SC,3.42,153051.0,-44081.0,133904.0,-5.80,-22.36,87.49,45.57,236.0,-0.33,1.08,8.51,XY24,NTT,AUTO
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.26,126403.0,1106.0,57817.0,-5.72,0.88,45.74,47.03,141.0,0.02,0.89,58.27,MH,ATH,POWER
70,SURYODAY,240.00,60.09,46.0,H-SC,15.67,146737.0,-32334.0,98064.0,-5.39,-18.06,66.83,36.71,135.0,-0.33,1.04,45.59,XR,NTT,BANKS
59,REPCOHOME,880.00,-53.80,62.0,H-SC,7.19,264751.0,-21754.0,279101.0,-4.84,-7.59,105.42,89.82,134.0,-0.08,1.87,40.07,XY24,NTT,FINANCE
64,SFL,1287.00,33.53,49.0,M-SC,4.31,176214.0,-86004.0,155826.0,-4.78,-32.80,88.43,26.63,229.0,-0.55,1.24,9.56,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.26,126403.0,1106.0,57817.0,-5.72,0.88,45.74,47.03,141.0,0.02,0.89,58.27,MH,ATH,POWER
56,RECLTD,446.00,42.86,46.0,M-LC,4.98,202262.0,-908.0,43041.0,-0.78,-0.45,21.28,20.74,55.0,-0.02,1.43,6.38,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,47.0,M-LC,10.51,206234.0,1158.0,111408.0,0.16,0.56,54.02,54.89,52.0,0.01,1.45,24.80,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.20,53.0,M-SC,1.76,87574.0,-13203.0,13294.0,-0.04,-13.10,15.18,0.09,245.0,-0.99,0.62,15.05,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.67,221400.0,-48267.0,84597.0,-1.02,-17.90,38.21,13.47,138.0,-0.57,1.56,12.58,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,43.0,H-SC,1.91,138034.0,-37869.0,79894.0,-4.66,-21.53,57.88,23.89,149.0,-0.47,0.97,19.21,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.52,97877.0,-29928.0,29911.0,0.02,-23.42,30.56,-0.01,232.0,-1.00,0.69,10.83,OX40N,NTT,AC
67,SIS,528.0,1988.08,38.0,H-SC,2.78,84278.0,-26754.0,50365.0,-0.15,-24.10,59.76,21.26,156.0,-0.53,0.59,13.87,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,69.0,M-LC,10.76,174416.0,33576.0,10918.0,1.23,23.84,6.26,31.59,66.0,3.08,1.23,56.64,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,53.17,64.0,M-MC,10.07,241838.0,16876.0,150786.0,1.64,7.50,62.35,74.53,192.0,0.11,1.71,42.17,XY24,BTT,STEEL
31,HINDZINC,730.22,26.60,47.0,M-LC,10.51,206234.0,1158.0,111408.0,0.16,0.56,54.02,54.89,52.0,0.01,1.45,24.80,X5K,ATH,METALS
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.26,126403.0,1106.0,57817.0,-5.72,0.88,45.74,47.03,141.0,0.02,0.89,58.27,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,69.0,M-LC,10.76,174416.0,33576.0,10918.0,1.23,23.84,6.26,31.59,66.0,3.08,1.23,56.64,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.31,123105.0,8307.0,85730.0,-2.64,7.24,69.64,81.92,223.0,0.10,0.87,45.74,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.31,123105.0,8307.0,85730.0,-2.64,7.24,69.64,81.92,223.0,0.10,0.87,45.74,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,52.0,H-SC,5.84,129378.0,6042.0,125587.0,-1.10,4.90,97.07,106.72,119.0,0.05,0.91,29.42,AR,ATH,MISC
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.26,126403.0,1106.0,57817.0,-5.72,0.88,45.74,47.03,141.0,0.02,0.89,58.27,MH,ATH,POWER
31,HINDZINC,730.22,26.60,47.0,M-LC,10.51,206234.0,1158.0,111408.0,0.16,0.56,54.02,54.89,52.0,0.01,1.45,24.80,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,37.0,M-LC,5.54,146729.0,-4216.0,113671.0,-0.53,-2.79,77.47,72.51,53.0,-0.04,1.04,3.06,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.44,25.0,X-SC,6.94,87465.0,-41205.0,86503.0,-1.15,-32.02,98.90,35.20,219.0,-0.48,0.62,0.39,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-24.37,26.0,X-MC,4.67,189743.0,-10604.0,41098.0,-2.97,-5.29,21.66,15.23,178.0,-0.26,1.34,25.25,X40N,NTT,REALTY
23,DMART,5391.45,-21.31,26.0,X-LC,8.24,200430.0,-9977.0,69148.0,-4.16,-4.74,34.50,28.12,38.0,-0.14,1.41,17.76,X40N,ATH,FMCG
30,HINDUNILVR,2922.00,-14.10,30.0,X-LC,6.84,241390.0,-15673.0,50813.0,-1.30,-6.10,21.05,13.67,24.0,-0.31,1.70,13.07,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.21,35.0,X-MC,0.91,198945.0,-1875.0,69671.0,-1.03,-0.93,35.02,33.76,80.0,-0.03,1.40,4.21,X40,ATH,FMCG
41,ITC,452.00,-38.77,43.0,X-LC,1.14,197996.0,-2142.0,23027.0,-0.98,-1.07,11.63,10.44,4.0,-0.09,1.40,4.43,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,38.0,H-LC,1.21,151905.0,-34190.0,81664.0,-2.61,-18.37,53.76,25.51,15.0,-0.42,1.07,11.94,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.87,35.0,X-MC,3.09,303870.0,-24121.0,130664.0,-2.65,-7.35,43.00,32.48,92.0,-0.18,2.14,3.23,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.09,67.0,X-LC,4.11,246270.0,12126.0,36497.0,-1.32,5.18,14.82,20.77,86.0,0.33,1.74,12.46,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,34.0,X-SC,37.73,49507.0,-15499.0,166428.0,-1.65,-23.84,336.17,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.44,25.0,X-SC,6.94,87465.0,-41205.0,86503.0,-1.15,-32.02,98.90,35.20,219.0,-0.48,0.62,0.39,X40,NTT,FOOTWEAR
51,PAGEIND,51769.92,-29.72,32.0,X-MC,7.25,79380.0,-4130.0,24163.0,-1.11,-4.95,30.44,23.98,82.0,-0.17,0.56,0.59,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.18,39.0,X-MC,7.36,216640.0,-46725.0,156046.0,-1.11,-17.74,72.03,41.51,84.0,-0.30,1.53,1.66,XY25,ATH,FMCG
1,ABB,7934.00,-42.30,36.0,H-LC,5.26,240960.0,-20659.0,139877.0,-4.27,-7.90,58.05,45.57,7.0,-0.15,1.70,2.53,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,34.0,X-SC,37.73,49507.0,-15499.0,166428.0,-1.65,-23.84,336.17,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.92,-29.72,32.0,X-MC,7.25,79380.0,-4130.0,24163.0,-1.11,-4.95,30.44,23.98,82.0,-0.17,0.56,0.59,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.17,42.0,X-SC,4.30,81517.0,-63643.0,139573.0,0.09,-43.84,171.22,52.31,136.0,-0.46,0.58,9.11,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-12.47,33.0,X-MC,2.69,86550.0,-4128.0,19032.0,-1.74,-4.55,21.99,16.44,101.0,-0.22,0.61,14.70,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.44,25.0,X-SC,6.94,87465.0,-41205.0,86503.0,-1.15,-32.02,98.90,35.20,219.0,-0.48,0.62,0.39,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-29.65,44.0,X-LC,9.89,280449.0,-65507.0,132204.0,-0.22,-18.94,47.14,19.28,1.0,-0.50,1.98,3.68,X40,ATH,IT
39,INFY,2275.00,-19.24,51.0,X-LC,5.11,314835.0,1947.0,169728.0,0.69,0.62,53.91,54.87,3.0,0.01,2.22,8.96,X40,BTT,IT
41,ITC,452.00,-38.77,43.0,X-LC,1.14,197996.0,-2142.0,23027.0,-0.98,-1.07,11.63,10.44,4.0,-0.09,1.40,4.43,X40,NTT,FMCG
83,VBL,671.64,-16.88,53.0,X-LC,6.90,299605.0,-16237.0,128890.0,-0.30,-5.14,43.02,35.67,5.0,-0.13,2.11,8.00,X40N,ATH,FMCG
1,ABB,7934.00,-42.30,36.0,H-LC,5.26,240960.0,-20659.0,139877.0,-4.27,-7.90,58.05,45.57,7.0,-0.15,1.70,2.53,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,34.0,L-SC,27.73,74458.0,-39091.0,79141.0,-4.18,-34.43,106.29,35.27,268.0,-0.49,0.53,82.24,XR,NTT,HOTELS
7,ASIANTILES,137.00,7327.78,63.0,L-SC,10.55,81507.0,-12303.0,88924.0,-2.43,-13.11,109.10,81.67,269.0,-0.14,0.57,57.84,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,50.0,H-SC,8.37,92100.0,-5880.0,27483.0,-1.85,-6.00,29.84,22.05,152.0,-0.21,0.65,33.87,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.31,123105.0,8307.0,85730.0,-2.64,7.24,69.64,81.92,223.0,0.10,0.87,45.74,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.26,126403.0,1106.0,57817.0,-5.72,0.88,45.74,47.03,141.0,0.02,0.89,58.27,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,72.45,63.0,H-SC,2.27,149742.0,-33033.0,144097.0,-3.30,-18.07,96.23,60.77,125.0,-0.23,1.06,34.12,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-30.74,70.0,M-SC,6.87,86196.0,-14963.0,54967.0,1.81,-14.79,63.77,39.54,202.0,-0.27,0.61,21.55,AR,ATH,AUTO
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.26,126403.0,1106.0,57817.0,-5.72,0.88,45.74,47.03,141.0,0.02,0.89,58.27,MH,ATH,POWER
5,ANGELONE,3033.00,20.92,68.0,X-SC,5.85,214036.0,23030.0,34674.0,3.64,12.06,16.20,30.21,157.0,0.66,1.51,35.46,X40N,NTT,FINANCE
62,SAMMAANCAP,326.00,-196.59,70.0,M-SC,33.08,167706.0,18486.0,125696.0,0.06,12.39,74.95,96.62,208.0,0.15,1.18,81.49,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.69
1,25,44.37
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.84
LC    33.10
MC    23.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.01
X40      16.27
X40N     12.48
XY25     11.93
XR        9.88
AR        7.79
OX40N     6.50
X200      1.77
X5K       1.45
SR        1.04
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.20
X-LC    22.43
X-MC    18.71
M-SC    12.62
X-SC     5.59
M-LC     5.15
H-LC     4.54
H-MC     2.33
M-MC     1.71
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-4.72,38.58
IT,12.45,-19.35,83.29
FINANCE,12.22,-9.11,58.57
MISC,7.80,-17.36,76.56
ELECTRICAL,5.67,-12.25,52.04
PAINTS,5.58,-16.75,34.16
BANKS,4.05,-17.90,86.48
INSURANCE,3.90,0.03,37.24
AUTO,2.73,-48.60,110.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3319558.0,22
XR,1258940.0,13
AR,1184468.0,9
X40,851142.0,12
X40N,697908.0,9
OX40N,562436.0,9
XY25,481838.0,8
SR,274047.0,2
X5K,111408.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3401934.0,24
M-SC,1255346.0,15
X-MC,1220835.0,14
X-LC,1099692.0,13
X-SC,556721.0,6
H-LC,295370.0,3
M-LC,279038.0,4
H-MC,258514.0,2
L-SC,257949.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193353.0      6
           AR         840504.0      5
           XR         785434.0      7
M-SC       XY24       726015.0      6
X-MC       XY24       574717.0      4
X-LC       X40        464743.0      5
           XY24       314151.0      2
X-SC       X40N       303790.0      4
X-MC       X40        299896.0      6
H-SC       SR         274047.0      2
           OX40N      250779.0      3
X-LC       X40N       234535.0      3
H-LC       AR         221541.0      2
X-MC       XY25       186639.0      2
H-MC       XY24       183190.0      1
L-SC       XR         168065.0      2
X-SC       XY24       166428.0      1
X-MC       X40N       159583.0      2
M-MC       XY24       150786.0      1
M-SC       OX40N      146449.0      4
           XR         134763.0      2
           XY25       125696.0      1
           AR         122423.0      2
M-LC       XR         113671.0      1
           X5K        111408.0      1
L-SC       OX40N       89884.0      1
X-SC       X40         86503.0      1
X-LC       XY25        86263.0      3
H-MC       OX40N       75324.0      1
H-LC       X200        73829.0      1
H-SC       MH          57817.0      1
L-MC       XR          57007.0      1
M-LC       XY25        43041.0      1
L-LC       XY25        40199.0      1
M-LC       XY24        10918.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
